# Лабораторная работа 3 

# Классификация с использованием BERT


### Бобряков А.С.

## Введение

### Постановка задачи

В этом задании вы будете классифицировать пары вопросов из stack overflow на предмет дубликатов.
Чтобы получить гораздо более высокое качество на гораздо меньшем количестве данных, чем DSSM, предлагается дообучать предобученную модель BERT.

### Библиотеки

Для этого задания вам понадобятся следующие библиотеки:
 - [Pytorch](https://pytorch.org/).
 - [Transformers](https://github.com/huggingface/transformers).
 
### Данные

Данные лежат в архиве task3_data.zip, который состоит из:

* train.tsv - обучающая выборка. В каждой строке записаны: <вопрос 1>, <вопрос 2>, <таргет>

* validation.tsv - dev выборка, которую можно использовать для подбора гиперпарамеров; например, для ранней остановки. В каждой строке через табуляцию записаны: , <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...

* test.tsv - тестовая выборка, по которой оценивается итоговое качество. В каждой строке через табуляцию записаны: , <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...

Скачать данные можно здесь: [ссылка на google диск](https://drive.google.com/file/d/1Owb5Vpv7mVjksYo7gD9VuHkMETkzhIdr/view?usp=sharing)

In [ ]:
# данные -> bert -> обучить получив CLS токены

In [1]:
import numpy as np

# Суть BERT
# подается ембеддинг преложения  -> затираются (маскируются) слова -> bert восстанавливает слова
def test_encode(encode):
    result = encode('this is some text', 'this is another text')

    assert result['input_ids'] == [101, 2023, 2003, 2070, 3793, 102, 2023, 2003, 2178, 3793, 102], \
        'input_ids should be [101, 2023, 2003, 2070, 3793, 102, 2023, 2003, 2178, 3793, 102]'
    assert result['token_type_ids'] == [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1], \
        'token_type_ids should be [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1]'
    
def test_dataset(dataset):
    assert len(dataset[0]) == 3, 'Dataset[idx] should output tuple with 3 elements.'
    assert isinstance(dataset[0][-1], np.int64) or isinstance(dataset[0][-1], int), \
        'target should np.int64 or int'
    
def test_collator(dataset, collate_fn):
    ids, token_type_ids, labels = collate_fn([dataset[i] for i in range(10)])
    assert ids.shape[0] == labels.shape[0] == token_type_ids.shape[0], \
        'ids, token_type_ids, labels shoud have equal first dimension'
    assert ids.shape[1] == token_type_ids.shape[1], 'Incorrect shape of ids or token_type_ids'
    
def test_model(dataloader, model, device):
    input_ids, token_type_ids, _ = map(lambda x: x.to(device), next(iter(dataloader)))
    pred_shape = model(input_ids, token_type_ids).shape
    assert len(pred_shape) == 1 and pred_shape[0] == input_ids.shape[0], \
        f'Incorrect shape for the output of the model: {pred_shape} instead of {[input_ids.shape[0]]}'

## Часть 1. Подготовка данных (1 балл)

Мы будем работать с теми же данными, которые были в первом задании. А также будем учиться классифицировать пары вопросов аналогично третьей части в первом задании. Теперь выборка для обучения сгенерирована заранее :)

In [2]:
import pandas as pd
import numpy as np

import tqdm
import os
import tests

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


Путь к папке с данными:

In [4]:
DATA_PATH = '/content/drive/My Drive/Colab Notebooks/Texts_2020/'

Считывание данных для обучения:

In [5]:
train = pd.read_table(os.path.join(DATA_PATH, 'train.tsv'))

Модель **BERT** использует специальный токенизатор Wordpiece для разбиения предложений на токены. Готовая предобученная версия такого токенизатора существует в библиотеке **transformers**. Есть два класса: **BertTokenizer** и **BertTokenizerFast**. Использовать можно любой, но второй вариант работает существенно быстрее.

Токенизаторы можно обучать с нуля на своем корпусе данных, а можно подгружать уже готовые. Готовые токенизаторы, как правило, соответствуют предобученной конфигурации модели, которая использует словарь из этого токенизатора. 

Мы будем использовать базовую конфигурацию предобученного **BERT** для модели и токенизатора:

In [6]:
BERT_MODEL = 'bert-base-uncased'

Подгружение предобученных моделей и токенизаторов в **huggingface** происходит с помощью конструктора **from_pretrained**.

В данном конструкторе можно указать либо путь к предобученному токенизатору, либо название предобученной конфигурации, как в нашем случае: тогда **transformers** сам подгрузит нужные параметры.

In [7]:
!pip install transformers

     |████████████████████████████████| 1.3MB 9.4MB/s eta 0:00:01
     |████████████████████████████████| 1.1MB 51.0MB/s 
     |████████████████████████████████| 2.9MB 54.5MB/s 
     |████████████████████████████████| 890kB 48.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=a1cdc29956c33fa8c21476c041a3e8ec93f13eb4e7499f952ba1303653eef7ee
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [8]:
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained(BERT_MODEL)

Для классификации пар предложений необходимо привести примеры к виду: 

**[CLS] sent 1 [SEP] sent2 [SEP]**, 

где последний [SEP] можно опустить - в некоторых реализациях его используют, в некоторых нет. Существенного влияния на качество он не оказывает.

Предлагается привести все предложения из обучения к данному виду перед созданием Dataset. Для этого удобно использовать метод **tokenizer.encode_plus**, который сам вставляет специальные специальные токены [CLS], [SEP] в числовое представление примера. 

Кроме того, данный метод сразу формирует для наших примеров сегментные эмбеддинги - т.е. сопоставляет всем токенам первого предложения эмбеддинг **А**, и всем токенам второго предложения эмбеддинг **Б**.

In [9]:
def encode(query1, query2):
    """
    Args:
        query1: query text
        query2: second query text
        
    Returns:
        obj: dict {'input_ids': [0, 1, 2, 2, 1], 'token_type_ids': [0, 0, 1, 1, 1]}
    """
    return tokenizer.encode_plus(query1, query2)

test_encode(encode)

In [10]:
tqdm.tqdm.pandas()

train['enc'] = train.progress_apply(lambda x: encode(x['question_1'], x['question_2']), axis=1)

100%|██████████| 600000/600000 [01:38<00:00, 6102.97it/s]


Проанализируйте количество токенов в получившихся представлениях объектов, выберите максимальный порог длины, затем обрежьте все представления по этому порогу. Это необходимо для более разумного использования видеопамяти.

**hint:** можно использовать квантиль из **np.percentile**

In [11]:
from tqdm.notebook import tqdm

lens = []
for e in tqdm(train['enc']):
    tti = e['token_type_ids']
    lens.append(sum(tti) - 1)
    lens.append(len(tti) - sum(tti) - 2)

In [12]:
lens = np.array(lens)

In [13]:
MAXLEN = int(np.percentile(lens, 99))

for e in tqdm(train['enc']):
    ii = e['input_ids']
    tti = e['token_type_ids']
    am = e['attention_mask']
    len1 = sum(tti) - 1
    len0 = len(tti) - len1 - 3
    if len1 > MAXLEN:
        ii = ii[:len0+2+MAXLEN] + [ii[-1]]
        tti = tti[:len0+2+MAXLEN] + [tti[-1]]
        am = am[:len0+2+MAXLEN] + [am[-1]]
    if len0 > MAXLEN:
        ii = ii[:MAXLEN+1] + ii[len0+1:]
        tti = tti[:MAXLEN+1] + tti[len0+1:]
        am = am[:MAXLEN+1] + am[len0+1:]

## Часть 2. Задание пайплайна обучения (2 балла)

**Внимание**. За эту часть можно получить ненулевой балл, только при демонстрации того, что ваша модель хоть как-то обучается и  работает.

### Датасет и загрузчик

Создайте датасет, из которого **DataLoader** будет брать объекты для формирования батчей.

In [15]:
from torch.utils.data import Dataset
import torch

class MyDataset(Dataset):
    
    def __init__(self, corpus, targets):
        self.objects = [(e['input_ids'], e['token_type_ids']) for e in corpus]
        self.targets = targets

    def __len__(self):
        return len(self.targets)
    
    def __getitem__(self, idx):
        """
        Returns:
            obj: (input_ids, token_type_ids, target)
        """
        return self.objects[idx][0], self.objects[idx][1], self.targets[idx]

In [16]:
ds = MyDataset(train['enc'], train['target'])

test_dataset(ds)

Реализуйте технику динамического паддинга батчей, используя функцию **collate_fn**, которую можно передать как одноименный параметр в класс **DataLoader**.

**hint**: удобно использовать метод **torch.nn.utils.rnn**. Обратите особое внимание на параметр *batch_first*

In [17]:
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch, pad_idx=0):
    """
        Args:
            batch: list of objects
            pad_idx: padding idx
        Returns:
            padded ids, token_type_ids, labels
    """
    padded_ids = pad_sequence([torch.LongTensor(e[0]) for e in batch], batch_first=True, padding_value=pad_idx)
    token_type_ids = pad_sequence([torch.LongTensor(e[1]) for e in batch], batch_first=True, padding_value=pad_idx)
    labels = torch.LongTensor([e[2] for e in batch])
    return padded_ids, token_type_ids, labels

test_collator(ds, collate_fn)

In [18]:
from torch.utils.data import DataLoader

BATCH_SIZE = 64

dataloader = DataLoader(ds, collate_fn=collate_fn, batch_size=BATCH_SIZE, shuffle=True)

### Модель


В библиотеке **transformers** есть классы для модели BERT, уже настроенные под решение конкретных задач, с соответствующими головами классификации. Но гораздо более гибкий подход --- использовать энкодер BERT и, по необходимости, входной слой BERT.

Существует два способа задания модели:
* с помощью конфига **transformers.BertConfig**, в котором указываются все гиперпараметры модели
* с помощью подгрузки предобученной модели. Можно загружать как свои предобученные модели, указав путь, так и готовые предобученные модели, указав название конфигурации. В данном задании мы уже выбрали как модель базовую конфигурацию *BERT base*:

In [19]:
from transformers import BertModel

bert = BertModel.from_pretrained(BERT_MODEL)

Напишите модель-обертку, которая:
* принимает на вход название конфигурации (или путь к предобученной модели) и загружает как свой внутренний слой, обычно называемый *backbone* слоем
* создает голову для классификации
* при вызове метода **forward** использует векторное представление токена [CLS] с последнего слоя для классификации

На вход BERT принимает:
* input_ids --- непосредственно индексы ваших токенов в словаре
* attention_mask --- булеву маску со значениями FALSE для всех PAD_IDX токенов
* token_type_ids --- индексы принадлежности токена к 1 или 2 вопросу

**hint:** в статье про BERT авторы опустили следующий архитектурный момент - представление CLS токена используется для NSP задачи, но перед классификацией оно проходит через так называемый **pooler** слой - линейный слой с *tanh* в качестве функции активации, который сохраняет размерность (т.е. на выходе оставляет hidden size значений). Если вы хотите использовать выход именно *pooler* слоя, нужно использовать вектор, получаемый из энкодера как второй элемент кортежа.

In [20]:
from torch import nn
import torch

class BERTClassifier(nn.Module):

    def __init__(self, bert, n_classes=1):
        super().__init__()
        self.bert = bert
        self.linear1 = nn.Linear(768, 384)
        self.relu = nn.ReLU()
        self.drop = nn.Dropout(0.7)
        self.linear2 = nn.Linear(384, n_classes)
        
    @classmethod
    def from_pretrained(cls, path, n_classes=1):
        bert = BertModel.from_pretrained(path)
        return cls(bert, n_classes)
        
    def forward(self, input_ids, attention_mask=None, token_type_ids=None):
        """
            Args:
                input_ids: token ids, shape = [batch_size, sequence_length]
                attention_mask: masks out padding tokens, shape = [batch_size, sequence_length]
                token_type_ids: segmend ids, shape = [batch_size, sequence_length]
            Returns:
                predictions, shape [batch_size]
        """
        out = self.bert(input_ids, attention_mask, token_type_ids)  
        out = self.relu(self.linear1(out[1]))
        out = self.linear2(self.drop(out))
        return out.view(-1)

In [21]:
device = torch.device('cuda')

model = BERTClassifier.from_pretrained(BERT_MODEL, n_classes=1).to(device)

test_model(dataloader, model, device)

### Оптимизатор

Для оптимизации **BERT** будем использовать **AdamW** c увеличенным learning rate'ом для параметров головы-классификатора. 

Отличие **AdamW** от **Adam** заключается в более корректной реализации $l_2$ регуляризации, которая задается параметром **weight_decay** при инициализации.

Параметры необходимо объединить на три группы:

* параметры, которым нужен weight decay --- все параметры из backbone, кроме сдвигов (bias) и LayerNorm слоев.
* остальные парамеры из backbone
* параметры головы классификации, для которых мы будем задавать гораздо больший learning rate

Будем использовать **model.named_parameters()**, чтобы разделить параметры на три группы, исходя из названий слоев.

In [22]:
NO_DECAY = ['bias', 'LayerNorm.weight']

def is_backbone(name):
    if name.find('bert') == -1:
        return False
    else:
        return True

def needs_decay(name):
    if sum([name.find(NO_DECAY[i]) for i in range(len(NO_DECAY))]) == -len(NO_DECAY):
        return True
    else:
        return False

def get_optimizer(model, lr, weight_decay, head_lr):
    grouped_parameters = [
        {
            'params': [param for name, param in model.named_parameters() if is_backbone(name) and needs_decay(name)],
            'lr': lr,
            'weight_decay': weight_decay,
        },
        {
            'params': [param for name, param in model.named_parameters() if is_backbone(name) and not needs_decay(name)],
            'lr': lr,
            'weight_decay': 0.,
        },
        {
            'params': [param for name, param in model.named_parameters() if not is_backbone(name)],
            'lr': head_lr,
            'weight_decay': weight_decay,
        }
    ]

    optimizer = torch.optim.AdamW(grouped_parameters, lr, weight_decay=weight_decay)

    return optimizer

LEARNING_RATE = 3e-5
WEIGHT_DECAY = 1e-3
HEAD_LEARNING_RATE = 1e-3 

optimizer = get_optimizer(model, lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY, head_lr=HEAD_LEARNING_RATE)

### Scheduler


Также необходимо задать расписание для learning rate. Для **BERT** используется **linear warmup**. 

В **transformers** есть реализация **linear warmup** с помощью метода **transformers.get_linear_schedule_with_warmup**, в которой learning rate стартует с 0, и в течение **num_warmup_steps** линейно возрастает до значения, указанного в качестве стартового в оптимизаторе. Затем в течение **num_training_steps - num_warmup_steps** learning rate линейно падает до 0.

Используйте *dataloader.dataset* и *dataloader.batch_size*, чтобы рассчитать *num_training_steps* исходя из количества эпох. В случае нашей задачи одной эпохи должно быть достаточно для обучения модели.

В случае ограниченного количества видеопамяти может возникнуть ситуация, при которой батч нужного размера не влезает в видеокарту. Для таких ситуаций предлагается использовать аккумуляцию градиента - накапливание градиента в течение *accumulation_steps* с последующим шагом спуска. Т.е. делать *(loss / accumulation_steps).backward()* для каждого батча, и при этом каждые *accumulation_steps* шагов делать *optimizer.step()*.

При обучении количество шагов warmup выбирают либо как 10000 шагов, либо как 0.01% или 0.06% от всех шагов.

In [23]:
import transformers

def get_scheduler(optimizer, dataloader, n_epochs, accumulation_steps, warmup_percentage):
    num_training_steps = n_epochs*len(dataloader)/accumulation_steps
    num_warmup_steps = warmup_percentage*num_training_steps
    return transformers.get_linear_schedule_with_warmup(optimizer, num_warmup_steps, num_training_steps)

N_EPOCHS = 1
ACCUMULATION_STEPS = 4
WARMUP_PERCENTAGE = 0.06

scheduler = get_scheduler(
    optimizer, dataloader, n_epochs=N_EPOCHS, accumulation_steps=ACCUMULATION_STEPS, warmup_percentage=WARMUP_PERCENTAGE
)

Для проверки качества модели необходимо использовать подготовленный для задания **Evaluator**. Важный момент: при использовании, evaluator переводит модель в режим валидации: model.eval(). Во время обучения необходимо самостоятельно переключать ее на model.train() после каждого использования.

На вход evaluator принимает вашу модель и device (CUDA или CPU), на котором необходимо считать результаты моделирования. 

При использовании evaluator можно использовать BATCH_SIZE значительно большего размера, потому что отпадает необходимость считать градиенты для параметров.

In [24]:
import numpy as np
import tqdm

import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

class InferenceDataset(Dataset):
    
    def __init__(self, data):
        super().__init__()
        self.data = data
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        ids = torch.tensor(self.data[idx]['input_ids'], dtype=torch.long)
        type_ids = torch.tensor(self.data[idx]['token_type_ids'], dtype=torch.long)
        return ids, type_ids

    
def collate_fn(batch, pad_idx=0):
    ids, type_ids = map(lambda x: pad_sequence(x, batch_first=True, padding_value=pad_idx), zip(*batch))
    return ids, type_ids


def hits_count(dup_ranks, k):
    ranks = [rank <= k for rank in dup_ranks]
    return 0. if not ranks else np.mean(ranks)


def dcg_score(dup_ranks, k):
    vals = [1. / np.log2(1. + rank) for rank in dup_ranks if rank <= k]
    return 0. if not vals else np.sum(vals) / len(dup_ranks)


class Evaluator:

    def __init__(self, path, tokenizer, maxlen, batch_size, pad_idx=0, verbose=False):
        self.tokenizer = tokenizer
        self.maxlen = maxlen
        self.pad_idx = pad_idx
        
        data = []
        for line in open(path, encoding='utf-8'):
            data.append(line.strip().split('\t'))

        lengths = []
        prep_data = []
        for query, *docs in tqdm.notebook.tqdm(data, disable=not verbose, desc='Encoding text...'):
            for doc in docs:
                prep_data.append(self.encode(query, doc))
            lengths.append(len(docs))
        self.bounds = np.cumsum([0] + lengths)
        self.ids, prep_data = zip(*sorted(enumerate(prep_data), key=lambda x: len(x[1]['input_ids'])))

        ds = InferenceDataset(prep_data)
        self.dataloader = DataLoader(ds, batch_size, collate_fn=collate_fn)
        
    def __call__(self, model, device, verbose=False):
        model.to(device)
        model.eval()
        
        preds = []
        for batch in tqdm.notebook.tqdm(self.dataloader, disable=not verbose, desc='Computing predictions...'):
            input_ids, token_type_ids = map(lambda x: x.to(device), batch)
            attention_mask = input_ids != self.pad_idx
            with torch.no_grad():
                pred = model(input_ids, attention_mask, token_type_ids).cpu()
            preds.append(pred)
        preds = torch.cat(preds).numpy()
        
        _, preds = zip(*sorted(zip(self.ids, preds), key=lambda x: x[0]))
        
        rankings = []
        for i in range(len(self.bounds) - 1):
            rankings.append(
                list(np.argsort(-np.array(preds[self.bounds[i]:self.bounds[i + 1]]))).index(0) + 1
            )
            
        metrics = {
            'DCG': {f'DCG@{k}': dcg_score(rankings, k) for k in [1, 5, 10, 100, 500, 1000]},
            'Hits': {f'Hits@{k}': hits_count(rankings, k) for k in [1, 5, 10, 100, 500, 1000]}
        }
        
        return metrics
            
    def encode(self, query, doc):
        enc = self.tokenizer.encode_plus(query, doc, add_special_tokens=True)
        return {'input_ids': enc.input_ids[:self.maxlen], 'token_type_ids': enc.token_type_ids[:self.maxlen]}

In [25]:
evaluator = Evaluator(os.path.join(DATA_PATH, 'validation.tsv'), tokenizer, maxlen=MAXLEN, batch_size=1024)
metrics = evaluator(model, device, verbose=True)
metrics

{'DCG': {'DCG@1': 0.04,
  'DCG@10': 0.06551888197111005,
  'DCG@100': 0.15617523614281956,
  'DCG@1000': 0.2085514787564211,
  'DCG@5': 0.05,
  'DCG@500': 0.19783818892915925},
 'Hits': {'Hits@1': 0.04,
  'Hits@10': 0.11,
  'Hits@100': 0.58,
  'Hits@1000': 1.0,
  'Hits@5': 0.06,
  'Hits@500': 0.9}}

Данный **evaluator** предлагается использовать не только для оценки итогового качества, но также для вывода промежуточных результатов на dev сете в логи с помощью **torch.utils.tensorboard.SummaryWriter**.

Перед обучением необходимо создать объект данного класса, указав папку для записи логов.

Во время обучения через каждые $10000$ объектов необходимо записывать значения метрик в логи с помощью методов **writer.add_scalars**. Кроме того, необходимо записывать значение функционала ошибки на каждом батче во время обучения с помощью метода **writer.add_scalar**.

## Часть 3. Обучение модели (7 баллов)

Ниже предлагаются примерные значения гиперпараметров, приводящие к необходимым метрикам качества. Для подбора точных значений гиперпараметров предлагается использовать *dev set*.

**Гиперпараметры для обучения:**

* размер батча в $\{32, 64\}$
* клиппинг нормы градиента (используйте **torch.nn.utils.clip_grad_norm_**)
* шаг обучения в $\{$1e-5, 2e-5, 3e-5, 4e-5$\}$
* weight decay в $\{$1e-2, 1e-3, 1e-4$\}$
* warmup percentage в $\{0.01, 0.06\}$
* шаг обучения для головы-классификатора в $\{10, 50, 100\}$ раз больше, чем для остальных параметров

In [26]:
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter(log_dir=DATA_PATH+'Logs_v11')
loss_function = nn.BCEWithLogitsLoss()
all_losses = []
all_metrics = []
step = 0

In [27]:
import tqdm

model.train()
for epoch in range(N_EPOCHS):
    for i, (padded_ids, token_type_ids, labels) in tqdm.notebook.tqdm(enumerate(dataloader)):
        padded_ids = padded_ids.to(device)
        token_type_ids = token_type_ids.to(device)
        labels = labels.to(device)
        attention_mask = (padded_ids != 0)
        out = model(padded_ids, attention_mask, token_type_ids)
        loss = loss_function(out, labels.float()) 
        loss.backward()
        if (i+1)%ACCUMULATION_STEPS == 0:
            writer.add_scalar('Train_Loss', loss, step)
            all_losses.append(loss.detach())
            torch.nn.utils.clip_grad_norm_(model.parameters(), 3)
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
            step += 1
        if (i+1)%(10000//BATCH_SIZE) == 0:
            metrics = evaluator(model, device, verbose=False)
            writer.add_scalars('Train_Hits', metrics['Hits'], step)
            writer.add_scalars('Train_DCG', metrics['DCG'], step)
            all_metrics.append(metrics)
            model.train()
writer.close()

In [28]:
dev_metrics = evaluator(model, device, verbose=True)
dev_metrics

{'DCG': {'DCG@1': 0.6,
  'DCG@10': 0.7028480739595744,
  'DCG@100': 0.7181339867677023,
  'DCG@1000': 0.7308080429976717,
  'DCG@5': 0.6873044948707077,
  'DCG@500': 0.7298025574542892},
 'Hits': {'Hits@1': 0.6,
  'Hits@10': 0.82,
  'Hits@100': 0.9,
  'Hits@1000': 1.0,
  'Hits@5': 0.77,
  'Hits@500': 0.99}}

In [29]:
test_evaluator = Evaluator(os.path.join(DATA_PATH, 'test.tsv'), tokenizer, maxlen=MAXLEN, batch_size=1024)
test_metrics = test_evaluator(model, device, verbose=True)
test_metrics

{'DCG': {'DCG@1': 0.64,
  'DCG@10': 0.713827432291599,
  'DCG@100': 0.7381232754951706,
  'DCG@1000': 0.7507278888628509,
  'DCG@5': 0.7010310788673668,
  'DCG@500': 0.7485778040615503},
 'Hits': {'Hits@1': 0.64,
  'Hits@10': 0.79,
  'Hits@100': 0.9,
  'Hits@1000': 1.0,
  'Hits@5': 0.75,
  'Hits@500': 0.98}}

Задание будет засчитано на полный балл при *Hits@1* на *test set* больше $0.6$. Необходимо приложить логи из тензорборда, а также скриншот этих самых логов.